# Title
[]()

In [1]:
import tabula
tabula.environment_info()

Python version:
    3.11.4 | packaged by Anaconda, Inc. | (main, Jul  5 2023, 13:47:18) [MSC v.1916 64 bit (AMD64)]
Java version:
    java version "1.8.0_401"
Java(TM) SE Runtime Environment (build 1.8.0_401-b10)
Java HotSpot(TM) Client VM (build 25.401-b10, mixed mode, sharing)
tabula-py version: 2.9.0
platform: Windows-10-10.0.19045-SP0
uname:
    uname_result(system='Windows', node='X380Yoga-2023-08-22', release='10', version='10.0.19045', machine='AMD64')
linux_distribution: ('', '', '')
mac_ver: ('', ('', '', ''), '')


In [7]:

import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
# sys.path.append(r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\GHL-chat\src\app')
from silvhua import *

In [44]:
# set the option to wrap text within cells
pd.set_option('display.max_colwidth', 200)
# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Iteration 1

In [2]:
from tabula import read_pdf
import json
import re
import warnings
def parse_pdf_tolist(
    pdf_filenames, 
    filepath=r'C:\Users\silvh\OneDrive\Ginkgo\references', 
    pages='all', 
    # last_id_column='Major Heading', value_to_ignore=None,
    # blank_columns=None,
    ):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages)
            dfs_list.append(df_list)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            message = f'An error occurred on line {lineno} in {filename}: {error}.'
            print(message)
    return dfs_list
        # file_dfs = []
        # page1_columns = df_list[0].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True).to_list()
        # print('number of columns:', len(page1_columns))
        # if last_id_column in page1_columns:
        #     page1_columns_sorted = sorted(page1_columns)
        # else:
        #     previous_columns = dfs_list[index-1].columns
        #     print(f'\t\tNo headers found; using headers from previous document:', previous_columns)
        #     if len(df_list[page_number].columns) > len(previous_columns):
        #         n_cols_to_pad = len(df_list[page_number].columns) - len(previous_columns)
        #         new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
        #         page1_columns_sorted = sorted(page1_columns) + new_columns_padding
        #         page1_columns = page1_columns + new_columns_padding

        #     elif len(df_list[page_number].columns) < len(previous_columns):
        #         page1_columns = (previous_columns[:len(df_list[page_number].columns)].to_list())
        #         page1_columns_sorted = sorted(page1_columns)
        #     else:
        #         page1_columns_sorted = (dfs_list[index-1].columns.to_list())
        #     page1_columns = ['Unnamed: 0' if column=='index' else column for column in page1_columns]
        #     print(f'\t\t**Current df columns:',page1_columns)
        
        # for page_number in range(len(df_list)):
        #     print(f'\tpage {page_number+1}...')
        #     df_list[page_number].columns = df_list[page_number].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True)
        #     print('\t\tCurrent number of columns:', len(df_list[page_number].columns), len(page1_columns_sorted))
        #     print(f'\t\t{len(page1_columns_sorted)}, {len(page1_columns)}')
        #     if len(df_list[page_number].columns.to_list()) == len(page1_columns_sorted):
        #         if sorted(df_list[page_number].columns.to_list()) == page1_columns_sorted:
        #             pass
        #         else:
        #             df_list[page_number].columns = page1_columns 
        #             print('\t\tKeeping page 1 columns')
        #     else:
        #         if len(df_list[page_number].columns) > len(page1_columns_sorted):
        #             n_cols_to_pad = len(df_list[page_number].columns) - len(page1_columns_sorted)
        #             new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
        #             df_list[page_number].columns = page1_columns + new_columns_padding
        #             print('\t\tPadding columns')
        #         elif len(df_list[page_number].columns) < len(page1_columns_sorted):
        #             df_list[page_number].columns = page1_columns[:len(df_list[page_number].columns)]
        #             print('\t\tDropping columns')
        #         else:
        #             df_list[page_number].columns = page1_columns
        #         corrected_pages[file] = page_number+1
        #         print(f'\n\t**Fixing column parsing for page {page_number+1} of {file}**')
        #         print(f'\t\t{df_list[page_number].columns}')
        #     # Consolidate rows that were incorrectly split
        #     if (df_list[page_number]['Unnamed: 0'].isna().sum() > 0):
        #         # print('\tAttempting to consolidate rows that were incorrectly split: ', df_list[page_number].columns)
        #         print('\tAttempting to consolidate rows that were incorrectly split: ')
        #         try:
        #             df_list[page_number] = merge_rows(df_list[page_number], file)
        #         except:
        #             print(f'\t\tUnable to consolidate rows on page {page_number+1} of {file}')
        #             reference_dict['unparsed documents'] = file
        #             continue

        #     # Align the columns
        #     if value_to_ignore in df_list[page_number].iloc[0].values:
        #         value_to_ignore_column = df_list[page_number].iloc[0].values.tolist().index(value_to_ignore)
        #         print(f'\tRemoving {value_to_ignore} from column {value_to_ignore_column}')
        #         df_list[page_number].iloc[0, value_to_ignore_column:-1] = df_list[page_number].iloc[0, value_to_ignore_column+1:].values

        #     for column in blank_columns:
        #         try:
        #             if df_list[page_number][column].isna().sum() != len(df_list[page_number]): # 2023-03-30 22:45:
        #                 blank_column_index = df_list[page_number].columns.tolist().index(column)
        #                 print(f'\tShifting data from column {blank_column_index}({column})')
        #                 df_list[page_number].iloc[:,blank_column_index+1:] = df_list[page_number].iloc[:,blank_column_index:-1].values 
        #                 df_list[page_number].iloc[:,blank_column_index] = None
        #         except:
        #             pass

        #     file_dfs.append(df_list[page_number]) # 2023-03-30 23:04 Decrease indent
        # dfs_list.append(file_dfs) # SH 2023-04-01 20:58

pdf_filenames = ['1_2024-adult-compendium_1_2024.pdf']

dfs_list = parse_pdf_tolist(
    pdf_filenames, 
    filepath=r'C:\Users\silvh\OneDrive\Ginkgo\references', 
    pages='all')
dfs_list

Error importing jpype dependencies. Fallback to subprocess.
No module named 'jpype'


Extracting the following files:
1_2024-adult-compendium_1_2024.pdf


[[   Major Heading  Activity Code  MET Value  \
  0      Bicycling           1003       14.0   
  1      Bicycling           1004       16.0   
  2      Bicycling           1008        8.5   
  3      Bicycling           1009        8.5   
  4      Bicycling           1010        4.0   
  5      Bicycling           1011        6.8   
  6      Bicycling           1013        5.8   
  7      Bicycling           1014        7.0   
  8      Bicycling           1015        4.3   
  9      Bicycling           1016        7.0   
  10     Bicycling           1017        9.0   
  11     Bicycling           1018        3.5   
  12     Bicycling           1019        5.8   
  13     Bicycling           1020        6.8   
  14     Bicycling           1030        8.0   
  15     Bicycling           1040       10.0   
  16     Bicycling           1050       12.0   
  17     Bicycling           1060       16.8   
  18     Bicycling           1065        8.5   
  19     Bicycling           1066       

In [17]:
len(dfs_list[0])

24

In [15]:
dfs_list[0][0]

,Major Heading,Activity Code,MET Value,Activity Description
0,Bicycling,1003,14.0,"Bicycling, mountain, uphill, vigorous"
1,Bicycling,1004,16.0,"Bicycling, mountain, competitive racing"
2,Bicycling,1008,8.5,"Bicycling, BMX"
3,Bicycling,1009,8.5,"Bicycling, mountain, general"
4,Bicycling,1010,4.0,"Bicycling, <10 mph, leisure, to work or for pl..."
5,Bicycling,1011,6.8,"Bicycling, to/from work, self selected pace"
6,Bicycling,1013,5.8,"Bicycling, on dirt or farm road, moderate pace"
7,Bicycling,1014,7.0,"Bicycling, general"
8,Bicycling,1015,4.3,"Bicycling, self-selected easy pace"
9,Bicycling,1016,7.0,"Bicycling, self-selected moderate pace"


In [28]:
dfs_list[0][3]

,Major Heading,Activity Code,MET Value,Activity Description
0,Fishing & Hunting,4020,4.0,Fishing from river bank and walking
1,Fishing & Hunting,4030,2.0,"Fishing from boat or canoe, sitting"
2,Fishing & Hunting,4040,3.5,"Fishing from river bank, standing (Taylor Code..."
3,Fishing & Hunting,4050,6.0,"Fishing in stream, in waders (Taylor Code 670)"
4,Fishing & Hunting,4060,2.0,"Fishing, ice, sitting"
5,Fishing & Hunting,4061,1.8,"Fishing, jig or line, standing, general"
6,Fishing & Hunting,4062,3.5,"Fishing, dip net, setting net and retrieving f..."
7,Fishing & Hunting,4063,3.8,"Fishing, set net, setting net and retrieving f..."
8,Fishing & Hunting,4064,3.0,"Fishing, fishing wheel, setting net and retrie..."
9,Fishing & Hunting,4065,2.3,"Fishing with a spear, standing"


# Iteration 2

In [22]:
from tabula import read_pdf
import json
import re
import warnings
def parse_pdf_tolist(
    pdf_filenames, 
    filepath=r'C:\Users\silvh\OneDrive\Ginkgo\references', 
    pages='all', 
    # last_id_column='Major Heading', value_to_ignore=None,
    # blank_columns=None,
    ):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages)
            dfs_list.append(df_list)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            message = f'An error occurred on line {lineno} in {filename}: {error}.'
            print(message)
    return dfs_list


def concat_parsed_pdfs(dfs_list):
    # Unpack the lists into a single unnested list
    if type(dfs_list[0]) != list:
        dfs_list = [dfs_list]
    unpacked_dfs_list = [item for sublist in dfs_list for item in sublist]
    print(f'Concatenating {len(unpacked_dfs_list)} dataframes...')
    page1_columns = unpacked_dfs_list[0].columns
    dfs_list = [unpacked_dfs_list[0]]
    n_rows = len(unpacked_dfs_list[0])
    for index, df in enumerate(unpacked_dfs_list):
        print(f'\tDataframe {index+1} (shape: {df.shape})...')
        df_columns = df.columns
        if page1_columns.equals(df_columns):
            pass
        elif len(page1_columns) == len(df_columns): 
            # If columns do not match, assume that the pages do not have headers and turn the dataframe into a row
            row_1_dict = {}
            for column, value in zip(page1_columns, df_columns):
                row_1_dict[column] = value
            df.columns = page1_columns
            dfs_list.append(pd.DataFrame(row_1_dict, index=[0]))
            n_rows += 1
        dfs_list.append(df)
        n_rows += len(df) 
    print(f'Total number of rows: {n_rows}')
    result_df = pd.concat(dfs_list, ignore_index=True)
    print(f'\tFinal dataframe shape: {result_df.shape}')
    return result_df

# pdf_filenames = ['1_2024-adult-compendium_1_2024.pdf']

# dfs_list = parse_pdf_tolist(
#     pdf_filenames, 
#     filepath=r'C:\Users\silvh\OneDrive\Ginkgo\references', 
#     pages='all')
# dfs_list

df = concat_parsed_pdfs(dfs_list)
df

Concatenating 24 dataframes...
	Dataframe 1 (shape: (41, 4))...
	Dataframe 2 (shape: (42, 4))...
	Dataframe 3 (shape: (42, 4))...
	Dataframe 4 (shape: (42, 4))...
	Dataframe 5 (shape: (43, 4))...
	Dataframe 6 (shape: (42, 4))...
	Dataframe 7 (shape: (42, 4))...
	Dataframe 8 (shape: (42, 4))...
	Dataframe 9 (shape: (46, 4))...
	Dataframe 10 (shape: (39, 4))...
	Dataframe 11 (shape: (38, 4))...
	Dataframe 12 (shape: (42, 4))...
	Dataframe 13 (shape: (39, 4))...
	Dataframe 14 (shape: (43, 4))...
	Dataframe 15 (shape: (41, 4))...
	Dataframe 16 (shape: (42, 4))...
	Dataframe 17 (shape: (42, 4))...
	Dataframe 18 (shape: (42, 4))...
	Dataframe 19 (shape: (42, 4))...
	Dataframe 20 (shape: (42, 4))...
	Dataframe 21 (shape: (40, 4))...
	Dataframe 22 (shape: (42, 4))...
	Dataframe 23 (shape: (42, 4))...
	Dataframe 24 (shape: (22, 4))...
Total number of rows: 1021
	Final dataframe shape: (1021, 4)


,Major Heading,Activity Code,MET Value,Activity Description
0,Bicycling,1003.0,14.0,"Bicycling, mountain, uphill, vigorous"
1,Bicycling,1004.0,16.0,"Bicycling, mountain, competitive racing"
2,Bicycling,1008.0,8.5,"Bicycling, BMX"
3,Bicycling,1009.0,8.5,"Bicycling, mountain, general"
4,Bicycling,1010.0,4.0,"Bicycling, <10 mph, leisure, to work or for pl..."
...,...,...,...,...
1016,Video Games,22200.0,3.0,"Active video game, motion sensing game/using u..."
1017,Video Games,22240.0,4.0,"Active video game, motion sensing game/using t..."
1018,Video Games,22280.0,5.0,"Active video game, motion sensing game/using t..."
1019,Video Games,22320.0,7.5,"Active video game, motion sensing game/using t..."


# Iteration 3

In [96]:
from tabula import read_pdf
def parse_pdf_tolist(
    pdf_filenames, 
    filepath=r'C:\Users\silvh\OneDrive\Ginkgo\references', 
    pages='all', 
    # last_id_column='Major Heading', value_to_ignore=None,
    # blank_columns=None,
    ):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages)
            dfs_list.append(df_list)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            message = f'An error occurred on line {lineno} in {filename}: {error}.'
            print(message)
    return dfs_list


def concat_parsed_pdfs(dfs_list):
    # Unpack the lists into a single unnested list
    if type(dfs_list[0]) != list:
        dfs_list = [dfs_list]
    unpacked_dfs_list = [item for sublist in dfs_list for item in sublist]
    print(f'Concatenating {len(unpacked_dfs_list)} dataframes...')
    page1_columns = unpacked_dfs_list[0].columns
    dfs_list = [unpacked_dfs_list[0]]
    n_rows = len(unpacked_dfs_list[0])
    print(f'\tDataframe 1 (shape: {unpacked_dfs_list[0].shape})...')
    for index, df in enumerate(unpacked_dfs_list[1:]):
        print(f'\tDataframe {index+2} (shape: {df.shape})...')
        df_columns = df.columns
        print(f'Dataframe {index+2} columns: {df_columns}')
        if page1_columns.equals(df_columns):
            pass
        elif len(page1_columns) == len(df_columns): 
            # If columns do not match, assume that the pages do not have headers and turn the dataframe into a row
            row_1_dict = {}
            for column, value in zip(page1_columns, df_columns):
                row_1_dict[column] = value
            df.columns = page1_columns
            dfs_list.append(pd.DataFrame(row_1_dict, index=[0]))
            n_rows += 1
        dfs_list.append(df)
        n_rows += len(df) 
    print(f'Total number of rows: {n_rows}')
    result_df = pd.concat(dfs_list, ignore_index=True)
    print(f'\tFinal dataframe shape: {result_df.shape}')
    return result_df

pdf_filenames = ['1_2024-adult-compendium_1_2024.pdf']

dfs_list = parse_pdf_tolist(
    pdf_filenames, 
    filepath=r'C:\Users\silvh\OneDrive\Ginkgo\references', 
    pages='all')

df = concat_parsed_pdfs(dfs_list[:2])
df

Extracting the following files:
1_2024-adult-compendium_1_2024.pdf
Concatenating 24 dataframes...
	Dataframe 1 (shape: (41, 4))...
	Dataframe 2 (shape: (42, 4))...
Dataframe 2 columns: Index(['Bicycling', '01270', '9.0',
       'Bicycling, stationary, RPM/Spin bike class'],
      dtype='object')
	Dataframe 3 (shape: (42, 4))...
Dataframe 3 columns: Index(['Conditioning Exercise', '02074', '14.0',
       'Rowing, stationary, ≥ 200 watts, very vigorous effort'],
      dtype='object')
	Dataframe 4 (shape: (42, 4))...
Dataframe 4 columns: Index(['Fishing & Hunting', '04010', '4.3',
       'Fishing related, digging worms, with shovel'],
      dtype='object')
	Dataframe 5 (shape: (43, 4))...
Dataframe 5 columns: Index(['Home Activities', '05026', '3.3',
       'Multiple household tasks all at once, moderate effort'],
      dtype='object')
	Dataframe 6 (shape: (42, 4))...
Dataframe 6 columns: Index(['Home Repair', '06100', '5.0', 'Cleaning gutters'], dtype='object')
	Dataframe 7 (shape: (42, 

,Major Heading,Activity Code,MET Value,Activity Description
0,Bicycling,1003,14.0,"Bicycling, mountain, uphill, vigorous"
1,Bicycling,1004,16.0,"Bicycling, mountain, competitive racing"
2,Bicycling,1008,8.5,"Bicycling, BMX"
3,Bicycling,1009,8.5,"Bicycling, mountain, general"
4,Bicycling,1010,4.0,"Bicycling, <10 mph, leisure, to work or for pleasure (Taylor Code 115)"
...,...,...,...,...
998,Video Games,22200,3.0,"Active video game, motion sensing game/using upper body (light to moderate effort)"
999,Video Games,22240,4.0,"Active video game, motion sensing game/using total body (moderate effort)"
1000,Video Games,22280,5.0,"Active video game, motion sensing game/using total body (moderate-to-vigorous effort)"
1001,Video Games,22320,7.5,"Active video game, motion sensing game/using total body, exergames, workouts, dancing (vigorous effort)"


In [32]:
from wrangling import *
filter = {
    'Activity Code': [15135]
}
filter_df_all_conditions(df, filter)

Results where ALL conditions are met:
	DataFrame indices: [576]
	DataFrame shape: (1, 4)


,Major Heading,Activity Code,MET Value,Activity Description
576,Sports,15135.0,5.8,NaN


In [37]:
df.loc[576:580]

,Major Heading,Activity Code,MET Value,Activity Description
576,Sports,15135.0,5.8,NaN
577,NaN,NaN,NaN,"jacks, arcade games), moderate effort"
578,Sports,15138.0,6.0,"Cheerleading, gymnastic moves, competitive"
579,Sports,15140.0,4.0,"Coaching, football, soccer, basketball, baseball, swimming, etc."
580,Sports,15142.0,8.0,"Coaching, actively playing sport with players"


In [31]:
save_excel(
    df, 
    filename='1_2024-adult-compendium_1_2024',
    path=r'C:\Users\silvh\OneDrive\Ginkgo\references', 
    col_width={
        'Major Heading': 20,
        'Activity Code': 20,
        'MET Value': 20,
        'Activity Description': 50
    },
)

The file '1_2024-adult-compendium_1_2024.xlsx' does not exist in 'C:/Users/silvh/OneDrive/Ginkgo/references/'.
File saved: C:/Users/silvh/OneDrive/Ginkgo/references/1_2024-adult-compendium_1_2024.xlsx
Time completed: 2024-03-13 00:59:06.679831


,Major Heading,Activity Code,MET Value,Activity Description
0,Bicycling,1003,14.0,"Bicycling, mountain, uphill, vigorous"
1,Bicycling,1004,16.0,"Bicycling, mountain, competitive racing"
2,Bicycling,1008,8.5,"Bicycling, BMX"
3,Bicycling,1009,8.5,"Bicycling, mountain, general"
4,Bicycling,1010,4.0,"Bicycling, <10 mph, leisure, to work or for pl..."
...,...,...,...,...
998,Video Games,22200,3.0,"Active video game, motion sensing game/using u..."
999,Video Games,22240,4.0,"Active video game, motion sensing game/using t..."
1000,Video Games,22280,5.0,"Active video game, motion sensing game/using t..."
1001,Video Games,22320,7.5,"Active video game, motion sensing game/using t..."


In [38]:
filter = {
    'Major Heading': 'null'
}
filter_df_any_condition(df, filter)

Results where any condition is met:
	DataFrame indices: [64, 66, 184, 186, 199, 345, 347, 351, 353, 359, 361, 376, 378, 443, 445, 575, 577]
	DataFrame shape: (17, 4)


,Major Heading,Activity Code,MET Value,Activity Description
64,NaN,NaN,NaN,"Resistance (weight lifting - free weight, nautilus or universal-type), power lifting or body building, vigorous effort ("
66,NaN,NaN,NaN,210)
184,NaN,NaN,NaN,Cooking or food preparation - standing or sitting or in general (not broken into stand/walk comp
186,NaN,NaN,NaN,light effort
199,NaN,NaN,NaN,"Laundry, fold or hang clothes, put clothes in washer or dryer, packing suitcase, washing clothes"
345,NaN,NaN,NaN,"Custodial work, moderate effort (e.g., buffing floors with electric buffer, feathering or sweeping arena floors,"
347,NaN,NaN,NaN,"out the trash, vaccuming)"
351,NaN,NaN,NaN,"Farming, moderate effort (e.g., feeding animals, chasing cattle by walking and/or horseback, spreading manur"
353,NaN,NaN,NaN,crops)
359,NaN,NaN,NaN,"Farming, taking care of animals (e.g., grooming, brushing, shearing sheep, assisting with birthing, medical care,"


In [48]:
df.loc[571:580]

,Major Heading,Activity Code,MET Value,Activity Description
571,Sports,15118.0,10.8,"Boxing, punching bag, 180 b/min"
572,Sports,15120.0,7.8,"Boxing, sparring"
573,Sports,15125.0,9.3,"Boxing, simulated boxing round, exercise"
574,Sports,15130.0,7.0,Broomball
575,NaN,NaN,NaN,"Children’s games, adults playing (e.g., hopscotch, 4-square, do"
576,Sports,15135.0,5.8,NaN
577,NaN,NaN,NaN,"jacks, arcade games), moderate effort"
578,Sports,15138.0,6.0,"Cheerleading, gymnastic moves, competitive"
579,Sports,15140.0,4.0,"Coaching, football, soccer, basketball, baseball, swimming, etc."
580,Sports,15142.0,8.0,"Coaching, actively playing sport with players"


In [47]:
df.loc[440:448]

,Major Heading,Activity Code,MET Value,Activity Description
440,Occupation,11750.0,2.5,"Tailoring, machine sewing"
441,Occupation,11760.0,3.5,"Tailoring, pressing"
442,Occupation,11763.0,2.0,"Tailoring, weaving, light effort (e.g., finishing operations, washing, dyeing, inspecting cloth, counting yards, paperwork)"
443,NaN,NaN,NaN,"Tailoring, weaving, moderate effort (e.g., spinning and weaving operations, delivering boxes of yarn to spinners, loadi"
444,Occupation,11765.0,4.0,NaN
445,NaN,NaN,NaN,"warp bean, pinwinding, conewinding, warping, cloth clutting)"
446,Occupation,11766.0,6.5,"Truck driving, loading and unloading truck, tying down load, standing, walking and carrying heavy loads"
447,Occupation,11767.0,2.0,"Truck driving, delivery truck, taxi, shuttlebus, school bus"
448,Occupation,11770.0,1.3,"Typing, electric, manual or computer"


In [46]:
df.loc[373:381]

,Major Heading,Activity Code,MET Value,Activity Description
373,Occupation,11260.0,5.0,"Forestry, ax chopping, slow, 1.25 kg axe, 19 blows/min, moderate effort"
374,Occupation,11262.0,8.0,"Forestry, ax chopping, fast, 1.25 kg axe, 35 blows/min, vigorous effort"
375,Occupation,11264.0,5.0,"Forestry, moderate effort (e.g., sawing wood with power saw, weeding, hoeing)"
376,NaN,NaN,NaN,"Forestry, vigorous effort (e.g., barking, felling, or trimming trees, carrying or stacking logs, planting seeds, sawi"
377,Occupation,11266.0,8.5,NaN
378,NaN,NaN,NaN,hand )
379,Occupation,11370.0,4.8,Furriery
380,Occupation,11375.0,3.8,"Garbage collector, walking, dumping bins into truck, street cleaning"
381,Occupation,11378.0,1.8,"Hairstylist (e.g., plaiting hair, manicure, make up artist)"


In [45]:
df.loc[341:365]

,Major Heading,Activity Code,MET Value,Activity Description
341,Occupation,11115,2.5,"Cook, chef"
342,Occupation,11120,4.0,"Construction, outside, remodeling, new structures (e.g., roof repair, miscellaneous)"
343,Occupation,11124,2.3,"Construction, rebar, bar bending/fixing"
344,Occupation,11125.0,2.3,"Custodial work, light effort (e.g., cleaning sink and toilet, dusting, vacuuming, light cleaning)"
345,NaN,NaN,NaN,"Custodial work, moderate effort (e.g., buffing floors with electric buffer, feathering or sweeping arena floors,"
346,Occupation,11126.0,3.8,NaN
347,NaN,NaN,NaN,"out the trash, vaccuming)"
348,Occupation,11130.0,3.3,"Electrical work (e.g., hook up wire, tapping splicing); plumbing moved to11516"
349,Occupation,11135.0,1.8,"Engineer (e.g., mechanical or electrical)"
350,Occupation,11145.0,7.8,"Farming, vigorous effort (e.g., baling hay, cleaning barn), includes former code 11200"


In [42]:
df.loc[195:203]

,Major Heading,Activity Code,MET Value,Activity Description
195,Home Activities,5065.0,2.3,"Non-food shopping, with or without cart, standing or walking"
196,Home Activities,5070.0,1.8,Ironing
197,Home Activities,5080.0,1.3,"Knitting, sewing, light effort, wrapping presents, sitting"
198,Home Activities,5082.0,2.8,Sewing with a machine
199,NaN,NaN,NaN,"Laundry, fold or hang clothes, put clothes in washer or dryer, packing suitcase, washing clothes"
200,Home Activities,5090.0,2.3,NaN
201,NaN,NaN,NaN,light effort
202,Home Activities,5091.0,2.0,"Laundry, fold or hang clothes, sitting"
203,Home Activities,5092.0,4.0,"Laundry, moderate effort"


In [98]:
df.loc[180:190]

,Major Heading,Activity Code,MET Value,Activity Description
180,Home Activities,5045.0,6.0,"Butchering animals, large, vigorous effort"
181,Home Activities,5046.0,2.3,"Cutting and smoking fish, drying fish or meat"
182,Home Activities,5048.0,4.0,"Tanning hides, general"
183,Home Activities,5049.0,3.5,"Cooking or food preparation, moderate effort"
184,NaN,NaN,NaN,Cooking or food preparation - standing or sitting or in general (not broken into stand/walk comp
185,Home Activities,5050.0,2.0,NaN
186,NaN,NaN,NaN,light effort
187,Home Activities,5051.0,2.3,"Cooking or food preparation, walking"
188,Home Activities,5052.0,2.3,"Serving food, setting/cleaning table, implied walking or standing"
189,Home Activities,5053.0,2.5,Feeding household animals


In [40]:
df.loc[570:580]

,Major Heading,Activity Code,MET Value,Activity Description
570,Sports,15115.0,8.5,"Boxing, punching bag, 120 b/min"
571,Sports,15118.0,10.8,"Boxing, punching bag, 180 b/min"
572,Sports,15120.0,7.8,"Boxing, sparring"
573,Sports,15125.0,9.3,"Boxing, simulated boxing round, exercise"
574,Sports,15130.0,7.0,Broomball
575,NaN,NaN,NaN,"Children’s games, adults playing (e.g., hopscotch, 4-square, do"
576,Sports,15135.0,5.8,NaN
577,NaN,NaN,NaN,"jacks, arcade games), moderate effort"
578,Sports,15138.0,6.0,"Cheerleading, gymnastic moves, competitive"
579,Sports,15140.0,4.0,"Coaching, football, soccer, basketball, baseball, swimming, etc."


In [ ]:
df.loc[60:70].fillna()

# Iteration 4

In [114]:
from tabula import read_pdf
def parse_pdf_tolist(
    pdf_filenames, 
    filepath=r'C:\Users\silvh\OneDrive\Ginkgo\references', 
    pages='all',**kwargs
    # last_id_column='Major Heading', value_to_ignore=None,
    # blank_columns=None,
    ):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages, **kwargs)
            dfs_list.append(df_list)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            message = f'An error occurred on line {lineno} in {filename}: {error}.'
            print(message)
    return dfs_list


def concat_parsed_pdfs(dfs_list):
    # Unpack the lists into a single unnested list
    if type(dfs_list[0]) != list:
        dfs_list = [dfs_list]
    unpacked_dfs_list = [item for sublist in dfs_list for item in sublist]
    print(f'Concatenating {len(unpacked_dfs_list)} dataframes...')
    page1_columns = unpacked_dfs_list[0].columns
    dfs_list = [unpacked_dfs_list[0]]
    n_rows = len(unpacked_dfs_list[0])
    print(f'\tDataframe 1 (shape: {unpacked_dfs_list[0].shape})...')
    for index, df in enumerate(unpacked_dfs_list[1:]):
        print(f'\tDataframe {index+2} (shape: {df.shape})...')
        df_columns = df.columns
        print(f'Dataframe {index+2} columns: {df_columns}')
        if page1_columns.equals(df_columns):
            pass
        elif len(page1_columns) == len(df_columns): 
            # If columns do not match, assume that the pages do not have headers and turn the dataframe into a row
            row_1_dict = {}
            for column, value in zip(page1_columns, df_columns):
                row_1_dict[column] = value
            df.columns = page1_columns
            dfs_list.append(pd.DataFrame(row_1_dict, index=[0]))
            n_rows += 1
        dfs_list.append(df)
        n_rows += len(df) 
    print(f'Total number of rows: {n_rows}')
    result_df = pd.concat(dfs_list, ignore_index=True)
    print(f'\tFinal dataframe shape: {result_df.shape}')
    return result_df

pdf_filenames = ['1_2024-adult-compendium_1_2024.pdf']

dfs_list = parse_pdf_tolist(
    pdf_filenames, 
    filepath=r'C:\Users\silvh\OneDrive\Ginkgo\references', 
    pages='all',
    )

df = concat_parsed_pdfs(dfs_list[:2])
df

Extracting the following files:
1_2024-adult-compendium_1_2024.pdf


Concatenating 24 dataframes...
	Dataframe 1 (shape: (41, 4))...
	Dataframe 2 (shape: (42, 4))...
Dataframe 2 columns: Index(['Bicycling', '01270', '9.0',
       'Bicycling, stationary, RPM/Spin bike class'],
      dtype='object')
	Dataframe 3 (shape: (42, 4))...
Dataframe 3 columns: Index(['Conditioning Exercise', '02074', '14.0',
       'Rowing, stationary, ≥ 200 watts, very vigorous effort'],
      dtype='object')
	Dataframe 4 (shape: (42, 4))...
Dataframe 4 columns: Index(['Fishing & Hunting', '04010', '4.3',
       'Fishing related, digging worms, with shovel'],
      dtype='object')
	Dataframe 5 (shape: (43, 4))...
Dataframe 5 columns: Index(['Home Activities', '05026', '3.3',
       'Multiple household tasks all at once, moderate effort'],
      dtype='object')
	Dataframe 6 (shape: (42, 4))...
Dataframe 6 columns: Index(['Home Repair', '06100', '5.0', 'Cleaning gutters'], dtype='object')
	Dataframe 7 (shape: (42, 4))...
Dataframe 7 columns: Index(['Lawn & Garden', '08010', '5.5',

,Major Heading,Activity Code,MET Value,Activity Description
0,Bicycling,1003,14.0,"Bicycling, mountain, uphill, vigorous"
1,Bicycling,1004,16.0,"Bicycling, mountain, competitive racing"
2,Bicycling,1008,8.5,"Bicycling, BMX"
3,Bicycling,1009,8.5,"Bicycling, mountain, general"
4,Bicycling,1010,4.0,"Bicycling, <10 mph, leisure, to work or for pleasure (Taylor Code 115)"
...,...,...,...,...
998,Video Games,22200,3.0,"Active video game, motion sensing game/using upper body (light to moderate effort)"
999,Video Games,22240,4.0,"Active video game, motion sensing game/using total body (moderate effort)"
1000,Video Games,22280,5.0,"Active video game, motion sensing game/using total body (moderate-to-vigorous effort)"
1001,Video Games,22320,7.5,"Active video game, motion sensing game/using total body, exergames, workouts, dancing (vigorous effort)"


# Fix parsing issues

In [117]:

def join_2_cells_below(
        df, column_to_concat, columns_to_fill=['Major Heading', 'Activity Code', 'MET Value'], 
        fill_method='bfill'
        ):
    df = df.copy()
    df['shifted'] = df[column_to_concat].shift(-2)
    rows_to_fix_filter = df['Major Heading'].isna()
    df.loc[rows_to_fix_filter, f'{column_to_concat} fixed'] = df[rows_to_fix_filter]['Activity Description'] + ' ' + df[rows_to_fix_filter]['shifted']
    df['Check Parsing'] = df[f'{column_to_concat} fixed'].notna()
    df[f'{column_to_concat} fixed'].fillna(df[column_to_concat], inplace=True)
    df[f'{column_to_concat} fixed'].fillna(method=fill_method, inplace=True)
    df.drop_duplicates(
        subset=f'{column_to_concat} fixed', keep='first', inplace=True
        )
    df.drop(columns=[column_to_concat, 'shifted'], inplace=True)
    df.rename(columns={f'{column_to_concat} fixed': column_to_concat}, inplace=True)
    df[columns_to_fill] = df[columns_to_fill].fillna(method=fill_method)
    df.drop_duplicates(
        subset=columns_to_fill, keep='first', inplace=True
        )
    return df



series1 = df.loc[60:70]
join_2_cells_below(series1, 'Activity Description')


,Major Heading,Activity Code,MET Value,Activity Description,Check Parsing
60,Conditioning Exercise,2040.0,7.5,"Circuit training, including kettlebells, some aerobic movement with minimal rest, general, vigorous intensity",False
61,Conditioning Exercise,2045.0,3.5,CurvesTM exercise routines in women,False
62,Conditioning Exercise,2048.0,5.0,"Elliptical trainer, moderate effort",False
63,Conditioning Exercise,2049.0,9.0,"Elliptical trainer, vigorous effort",False
64,Conditioning Exercise,2050.0,6.0,"Resistance (weight lifting - free weight, nautilus or universal-type), power lifting or body building, vigorous effort ( 210)",True
67,Conditioning Exercise,2052.0,5.0,"Resistance (weight) training, squats, deadlift, slow or explosive effort",False
68,Conditioning Exercise,2054.0,3.5,"Resistance (weight) training, multiple exercises, 8-15 reps at varied resistance",False
69,Conditioning Exercise,2055.0,5.8,"Resistance Training, circuit, reciprocol supersets, peripheral hear action training",False
70,Conditioning Exercise,2056.0,3.0,"Body weight resistance exercises (e.g., squat, lunge, push-up, crunch), general",False


In [118]:
series1 = df.loc[570:580]
join_2_cells_below(series1, 'Activity Description')

,Major Heading,Activity Code,MET Value,Activity Description,Check Parsing
570,Sports,15115.0,8.5,"Boxing, punching bag, 120 b/min",False
571,Sports,15118.0,10.8,"Boxing, punching bag, 180 b/min",False
572,Sports,15120.0,7.8,"Boxing, sparring",False
573,Sports,15125.0,9.3,"Boxing, simulated boxing round, exercise",False
574,Sports,15130.0,7.0,Broomball,False
575,Sports,15135.0,5.8,"Children’s games, adults playing (e.g., hopscotch, 4-square, do jacks, arcade games), moderate effort",True
578,Sports,15138.0,6.0,"Cheerleading, gymnastic moves, competitive",False
579,Sports,15140.0,4.0,"Coaching, football, soccer, basketball, baseball, swimming, etc.",False
580,Sports,15142.0,8.0,"Coaching, actively playing sport with players",False


In [119]:
series1 = df.loc[180:190]
join_2_cells_below(series1, 'Activity Description')

,Major Heading,Activity Code,MET Value,Activity Description,Check Parsing
180,Home Activities,5045.0,6.0,"Butchering animals, large, vigorous effort",False
181,Home Activities,5046.0,2.3,"Cutting and smoking fish, drying fish or meat",False
182,Home Activities,5048.0,4.0,"Tanning hides, general",False
183,Home Activities,5049.0,3.5,"Cooking or food preparation, moderate effort",False
184,Home Activities,5050.0,2.0,Cooking or food preparation - standing or sitting or in general (not broken into stand/walk comp light effort,True
187,Home Activities,5051.0,2.3,"Cooking or food preparation, walking",False
188,Home Activities,5052.0,2.3,"Serving food, setting/cleaning table, implied walking or standing",False
189,Home Activities,5053.0,2.5,Feeding household animals,False
190,Home Activities,5054.0,3.5,"carrying groceries, on level ground, walking",False


In [121]:
fixed_df = join_2_cells_below(df, 'Activity Description')
fixed_df

,Major Heading,Activity Code,MET Value,Activity Description,Check Parsing
0,Bicycling,1003,14.0,"Bicycling, mountain, uphill, vigorous",False
1,Bicycling,1004,16.0,"Bicycling, mountain, competitive racing",False
2,Bicycling,1008,8.5,"Bicycling, BMX",False
3,Bicycling,1009,8.5,"Bicycling, mountain, general",False
4,Bicycling,1010,4.0,"Bicycling, <10 mph, leisure, to work or for pleasure (Taylor Code 115)",False
...,...,...,...,...,...
998,Video Games,22200,3.0,"Active video game, motion sensing game/using upper body (light to moderate effort)",False
999,Video Games,22240,4.0,"Active video game, motion sensing game/using total body (moderate effort)",False
1000,Video Games,22280,5.0,"Active video game, motion sensing game/using total body (moderate-to-vigorous effort)",False
1001,Video Games,22320,7.5,"Active video game, motion sensing game/using total body, exergames, workouts, dancing (vigorous effort)",False


# Categorize

In [157]:
import re

def rename_dict_keys(dictionary, old_key, new_key):
    dictionary[new_key] = dictionary.pop(old_key)
    return dictionary

def convert_dict_key_to_regex(dictionary):
    new_dict = {}
    for key, value in dictionary.items():
        new_key = re.compile(rf'(?i)^.*?{key}.*?$')
        new_dict[new_key] = value
    return new_dict
mapping_dict = {
    'video game': 'Fitness Video Game',
    'calisthenics': 'Calisthenics'
}

# rename_dict_keys(mapping_dict, 'video game', 'hello')

convert_dict_key_to_regex(mapping_dict)

{re.compile(r'(?i)^.*?video game.*?$',
            re.IGNORECASE|re.UNICODE): 'Fitness Video Game',
 re.compile(r'(?i)^.*?calisthenics.*?$',
            re.IGNORECASE|re.UNICODE): 'Calisthenics'}

In [205]:
def convert_dict_keys_to_regex(dictionary):
    new_dict = {}
    for key, value in dictionary.items():
        new_key = re.compile(rf'(?i)^.*?{key}.*?$')
        new_dict[new_key] = value
    return new_dict

def simplify_strings(
        df, mapping_dict, original_column='Activity Description', new_column='Simplified Name', regex=True):
    if regex == True:
        mapping_dict = convert_dict_keys_to_regex(mapping_dict)
    df[new_column] = df[original_column].replace(
        mapping_dict, regex=regex,
        )
    return df

def append_description(df, description_mapping_dict,
        source_column='Activity Description', column_to_modify='Simplified Name', 
        position='before', sep=' ', regex=True
        ):
    if regex == True:
        description_mapping_dict = convert_dict_keys_to_regex(description_mapping_dict)
    df['Description'] = df[source_column].replace(description_mapping_dict, regex=regex)
    # Remove Descriptions if they are identical to the source column
    df.loc[df[df['Description'] == df[source_column]].index, 'Description'] = ''
    if position == 'after':
        df[column_to_modify] = df[column_to_modify] + sep + df['Description']
    else:
        df[column_to_modify] = df['Description'] + sep + df[column_to_modify]
    df.drop(columns='Description', inplace=True)

    return df

def simplify_column_with_description(
        df, mapping_dict, description_mapping_dict, **kwargs
        ):
    simplified_df = simplify_strings(
        df, mapping_dict,regex=True
        )
    simplified_df = append_description(
        simplified_df, description_mapping_dict
    )
    return simplified_df
    
mapping_dict = {
    # r'(?i)^.*?bicycling.*?$': 'Biking',
    'bicycling': 'Biking',
    'video game': 'Fitness Video Game',
    'calisthenics': 'Calisthenics'
}
description_mapping_dict = {
    # 'bicycling': 'Biking',
    'stationary': 'Indoor',
    'mountain': 'Mountain'
}
simplify_column_with_description(df, mapping_dict, description_mapping_dict)

,Major Heading,Activity Code,MET Value,Activity Description,Simplified Name
0,Bicycling,1003,14.0,"Bicycling, mountain, uphill, vigorous",Mountain Biking
1,Bicycling,1004,16.0,"Bicycling, mountain, competitive racing",Mountain Biking
2,Bicycling,1008,8.5,"Bicycling, BMX",Biking
3,Bicycling,1009,8.5,"Bicycling, mountain, general",Mountain Biking
4,Bicycling,1010,4.0,"Bicycling, <10 mph, leisure, to work or for pleasure (Taylor Code 115)",Biking
...,...,...,...,...,...
998,Video Games,22200,3.0,"Active video game, motion sensing game/using upper body (light to moderate effort)",Fitness Video Game
999,Video Games,22240,4.0,"Active video game, motion sensing game/using total body (moderate effort)",Fitness Video Game
1000,Video Games,22280,5.0,"Active video game, motion sensing game/using total body (moderate-to-vigorous effort)",Fitness Video Game
1001,Video Games,22320,7.5,"Active video game, motion sensing game/using total body, exergames, workouts, dancing (vigorous effort)",Fitness Video Game


# *End of Page*